<a href="https://colab.research.google.com/github/mikegee404/NewsBot-Intelligence-System/blob/main/NewsBot_Intelligence_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install kaggle


In [24]:
from google.colab import files
print("Please upload your kaggle.json file:")
uploaded = files.upload()

Please upload your kaggle.json file:


Saving kaggle.json to kaggle (2).json


In [25]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [26]:
print("✅ Kaggle API setup complete!")

✅ Kaggle API setup complete!


In [27]:
!kaggle competitions download -c learn-ai-bbc

learn-ai-bbc.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!unzip learn-ai-bbc.zip

Archive:  learn-ai-bbc.zip
replace BBC News Sample Solution.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BBC News Test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BBC News Train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [29]:
!ls -la

total 10180
drwxr-xr-x 1 root root    4096 Nov  2 00:50  .
drwxr-xr-x 1 root root    4096 Nov  1 23:22  ..
-rw-r--r-- 1 root root   10369 Dec  2  2019 'BBC News Sample Solution.csv'
-rw-r--r-- 1 root root 1712432 Dec  2  2019 'BBC News Test.csv'
-rw-r--r-- 1 root root 3351206 Dec  2  2019 'BBC News Train.csv'
drwxr-xr-x 4 root root    4096 Oct 30 13:36  .config
-rw-r--r-- 1 root root      66 Nov  1 23:24 'kaggle (1).json'
-rw-r--r-- 1 root root      66 Nov  2 00:50 'kaggle (2).json'
-rw-r--r-- 1 root root      66 Nov  1 23:24  kaggle.json
-rw-r--r-- 1 root root 1936538 Dec  2  2019  learn-ai-bbc.zip
-rw-r--r-- 1 root root   21601 Nov  2 00:03  NewsBot_8Module_Notebook.ipynb
-rw-r--r-- 1 root root 3349718 Nov  1 23:40  newsbot_dataset.csv
drwxr-xr-x 1 root root    4096 Oct 30 13:36  sample_data


In [30]:
import pandas as pd
import os

In [31]:
print("Available files:")
for file in os.listdir('.'):
    if file.endswith('.csv'):
        print(f"  - {file}")

Available files:
  - newsbot_dataset.csv
  - BBC News Sample Solution.csv
  - BBC News Train.csv
  - BBC News Test.csv


In [32]:
df = pd.read_csv('BBC News Train.csv')  # Adjust filename based on what you see
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Categories: {df['Category'].unique()}")  # Adjust column name as needed

Dataset shape: (1490, 3)
Columns: ['ArticleId', 'Text', 'Category']
Categories: ['business' 'tech' 'politics' 'sport' 'entertainment']


In [33]:
import pandas as pd
import numpy as np

In [34]:
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
print(df.head())

Dataset Info:
Shape: (1490, 3)
Columns: ['ArticleId', 'Text', 'Category']

First few rows:
   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business


In [35]:
text_column = 'Text'  # or 'description', 'content', 'headline', etc.
category_column = 'Category'  # or 'label', 'class', etc.

In [36]:
print(f"\nMissing values:")
print(df.isnull().sum())


Missing values:
ArticleId    0
Text         0
Category     0
dtype: int64


In [37]:
df_clean = df.dropna(subset=[text_column, category_column])

In [38]:
print(f"\nCategory distribution:")
print(df_clean[category_column].value_counts())


Category distribution:
Category
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64


In [39]:
if len(df_clean) > 2000:
    df_final = df_clean.sample(n=2000, random_state=42)
    print(f"\nSampled dataset to {len(df_final)} rows")
else:
    df_final = df_clean


In [40]:
df_final = df_final.rename(columns={
    text_column: 'content',
    category_column: 'category'
})

In [41]:
df_final.to_csv('newsbot_dataset.csv', index=False)
print("\n✅ Dataset prepared and saved as 'newsbot_dataset.csv'")


✅ Dataset prepared and saved as 'newsbot_dataset.csv'


In [1]:
# ==============================
# 0) Setup & Imports (run first)
# ==============================
import os, re, json, math, string, itertools, collections, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Sentiment / Emotion
from nltk.sentiment import SentimentIntensityAnalyzer  # VADER

# Explicitly install compatible versions to avoid numpy compatibility issues
!pip uninstall numpy -y
!pip install numpy==1.26.4
!pip install spacy==3.7.2 thinc==8.2.3 --no-deps
!pip -q install nrclex

from nrclex import NRCLex

# spaCy (POS/Dependency/NER)
import spacy
!python -m spacy download en_core_web_sm -q
nlp = spacy.load("en_core_web_sm")

# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
gradio 5.49.1 requires typer<1.0,>=0.12, but you have typer 0.9.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>

KeyboardInterrupt: 

In [2]:
import os
import pandas as pd

# Flexible CSV loader: tries common names, otherwise shows what's available.
candidate_files = [
    "BBC News Train.csv", "train.csv", "bbc-text.csv", "bbc_train.csv",
    "bbc_articles.csv", "bbc-data.csv", "newsbot_dataset.csv"
]

csv_path = None
for f in candidate_files:
    if os.path.exists(f):
        csv_path = f
        break

if csv_path is None:
    print("No known CSV found. CSVs in this directory:")
    for f in os.listdir('.'):
        if f.lower().endswith('.csv'):
            print(" -", f)
    # Set csv_path manually if needed, e.g.:
    # csv_path = "YOUR_FILE.csv"
    if csv_path is None:
        raise SystemExit("Please set csv_path to your dataset filename and re-run.")

print("Using dataset:", csv_path)
df_raw = pd.read_csv(csv_path)

print("Columns:", df_raw.columns.tolist())

text_col_candidates = ['text','article','content','body','Text','Article','Content','Body']
label_col_candidates = ['category','label','Category','Label','topic','Topic','class','Class']

def pick_col(cands, cols):
    for c in cands:
        if c in cols:
            return c
    return None

TEXT_COL = pick_col(text_col_candidates, df_raw.columns)
LABEL_COL = pick_col(label_col_candidates, df_raw.columns)

if TEXT_COL is None or LABEL_COL is None:
    raise ValueError("Could not infer TEXT_COL or LABEL_COL. Please set them manually.")

df = df_raw[[TEXT_COL, LABEL_COL]].dropna().rename(columns={TEXT_COL:'text', LABEL_COL:'label'}).reset_index(drop=True)
df['label'] = df['label'].astype(str)

print("Detected text column as 'text' and label column as 'label'")
display(df.head(3))
print("Class distribution:\n", df['label'].value_counts())

Using dataset: BBC News Train.csv
Columns: ['ArticleId', 'Text', 'Category']
Detected text column as 'text' and label column as 'label'


,text,label
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business


Class distribution:
 label
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64


In [3]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download the stopwords corpus if not already present
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Download the punkt_tab resource if not already present
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')


url_pat = re.compile(r'https?://\S+|www\.\S+')
email_pat = re.compile(r'\S+@\S+')
handle_pat = re.compile(r'@\w+')
num_pat = re.compile(r'\b\d+\b')


lemmatizer = WordNetLemmatizer()

def clean_text(s):
    s = s if isinstance(s, str) else ""
    s = s.lower()
    s = url_pat.sub(' ', s)
    s = email_pat.sub(' ', s)
    s = handle_pat.sub(' ', s)
    s = num_pat.sub(' ', s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(s)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words and len(t) > 2]
    return " ".join(tokens)

df['text_clean'] = df['text'].apply(clean_text)
display(df[['text','text_clean']].head(3))

,text,text_clean
0,worldcom ex-boss launches defence lawyers defe...,worldcom exboss launch defence lawyer defendin...
1,german business confidence slides german busin...,german business confidence slide german busine...
2,bbc poll indicates economic gloom citizens in ...,bbc poll indicates economic gloom citizen majo...


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True
)

lr = LogisticRegression(max_iter=2000)

pipe = Pipeline([('tfidf', tfidf), ('clf', lr)])

param_grid = {
    'tfidf__ngram_range': [(1,1),(1,2)],
    'tfidf__min_df': [2,3,5],
    'clf__C': [0.5, 1.0, 2.0]
}

grid = GridSearchCV(pipe, param_grid, cv=3, n_jobs=-1, scoring='f1_macro', verbose=0)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Train macro-F1:", f1_score(y_train, grid.predict(X_train), average='macro'))
print("Test  macro-F1:", f1_score(y_test, grid.predict(X_test), average='macro'))

best_vec = grid.best_estimator_.named_steps['tfidf']
feature_names = np.array(best_vec.get_feature_names_out())
best_clf = grid.best_estimator_.named_steps['clf']

class_labels = best_clf.classes_
coefs = best_clf.coef_
top_k = 15

category_top_terms = {}
for idx, cls in enumerate(class_labels):
    top_idx = np.argsort(coefs[idx])[-top_k:]
    category_top_terms[cls] = list(feature_names[top_idx][::-1])

print("\nTop indicative terms per category:")
for k,v in category_top_terms.items():
    print(f"{k:>12}: {', '.join(v)}")

# Simple bar viz for one category
pick = class_labels[0]
terms = category_top_terms[pick]
scores = [coefs[list(class_labels).index(pick), np.where(feature_names==t)[0][0]] for t in terms]

plt.figure(figsize=(8,4))
plt.barh(terms[::-1], scores[::-1])
plt.title(f"Top TF-IDF terms indicative of '{pick}' (LogReg coefficients)")
plt.tight_layout()
plt.show()

Best params: {'clf__C': 2.0, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Train macro-F1: 0.9991502335524414
Test  macro-F1: 0.9752007884760445

Top indicative terms per category:
    business: firm, company, bank, market, share, economic, economy, price, analyst, investment, business, growth, profit, investor, oil
entertainment: film, star, show, music, singer, album, award, actor, band, song, chart, festival, oscar, pop, comedy
    politics: labour, party, blair, minister, election, tory, government, secretary, leader, tony blair, would, conservative, tony, prime minister, prime
       sport: game, champion, match, player, coach, win, club, cup, team, season, england, side, injury, chelsea, olympic
        tech: user, computer, technology, software, online, net, people, mobile, digital, microsoft, internet, service, use, phone, website


NameError: name 'plt' is not defined

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
import collections
import pandas as pd # Import pandas as it is used in display(pos_df.head(15))

N = min(1500, len(df))
docs = list(nlp.pipe(df['text'].iloc[:N].tolist(), disable=[]))

pos_counts = collections.defaultdict(lambda: collections.Counter())
for doc, label in zip(docs, df['label'].iloc[:N]):
    pos_counts[label].update([t.pos_ for t in doc if not t.is_space])

pos_df = pd.DataFrame(pos_counts).fillna(0).astype(int)
pos_df = pos_df.reindex(sorted(pos_df.index), axis=0)
print("POS counts (rows=POS, cols=category):")
display(pos_df.head(15))

style = {}
for cat in pos_df.columns:
    adj_adv = pos_df.loc[pos_df.index.isin(["ADJ","ADV"]), cat].sum()
    noun_verb = pos_df.loc[pos_df.index.isin(["NOUN","VERB"]), cat].sum()
    style[cat] = {"adj_adv_ratio": (adj_adv+1)/(noun_verb+1)}

style_df = pd.DataFrame(style).T.sort_values('adj_adv_ratio', ascending=False)
print("Style ratios (higher → more descriptive/adverbial):")
display(style_df)

In [ ]:

import collections

def syntactic_features(doc):
    feats = collections.Counter()
    for tok in doc:
        dep = tok.dep_.lower()
        if dep in ["nsubj","dobj","pobj","iobj","amod","advmod","compound","neg"]:
            feats[dep] += 1
    return feats

syn_rows = []
for doc, label in zip(docs, df['label'].iloc[:N]):
    feats = syntactic_features(doc)
    feats['label'] = label
    syn_rows.append(feats)

syn_df = pd.DataFrame(syn_rows).fillna(0)
print("Syntactic feature head:")
display(syn_df.head())

def extract_svo(doc):
    svos = []
    verbs = [t for t in doc if t.pos_ == "VERB"]
    for v in verbs:
        subs = [w for w in v.children if w.dep_ in ("nsubj","nsubjpass")]
        objs = [w for w in v.children if w.dep_ in ("dobj","attr","pobj")]
        for s in subs:
            for o in objs:
                svos.append((s.lemma_.lower(), v.lemma_.lower(), o.lemma_.lower()))
    return svos

# Example SVOs
for i in range(min(3, N)):
    print(f"\nDoc {i} label={df['label'].iloc[i]}")
    print(extract_svo(docs[i])[:10])


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the vader_lexicon if not already present
try:
    nltk.data.find('sentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt')
except LookupError:
    nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def vader_score(text):
    s = sia.polarity_scores(text)
    return s['compound']

df['sentiment'] = df['text'].apply(vader_score)

sent_stats = df.groupby('label')['sentiment'].agg(['count','mean','median','std']).sort_values('mean', ascending=False)
print("Sentiment by category (VADER compound):")
display(sent_stats)

# Emotions via NRCLex (sample for speed)
from nrclex import NRCLex
emo_rows = []
for i, row in df.head(300).iterrows():
    try:
        emo = NRCLex(row['text'])
        e = dict(emo.top_emotions)  # list of (emotion, score)
    except Exception:
        e = {}
    e['label'] = row['label']
    emo_rows.append(e)

emo_df = pd.DataFrame(emo_rows).fillna(0)
print("Emotion sample head:")
display(emo_df.head())

emo_means = emo_df.groupby('label').mean().sort_index()
print("Average emotions by category (sample):")
display(emo_means)

In [ ]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import os

# Flexible CSV loader: tries common names, otherwise shows what's available.
candidate_files = [
    "BBC News Train.csv", "train.csv", "bbc-text.csv", "bbc_train.csv",
    "bbc_articles.csv", "bbc-data.csv", "newsbot_dataset.csv"
]

csv_path = None
for f in candidate_files:
    if os.path.exists(f):
        csv_path = f
        break

if csv_path is None:
    print("No known CSV found. CSVs in this directory:")
    for f in os.listdir('.'):
        if f.lower().endswith('.csv'):
            print(" -", f)
    # Set csv_path manually if needed, e.g.:
    # csv_path = "YOUR_FILE.csv"
    if csv_path is None:
        raise SystemExit("Please set csv_path to your dataset filename and re-run.")

print("Using dataset:", csv_path)
df_raw = pd.read_csv(csv_path)

print("Columns:", df_raw.columns.tolist())

text_col_candidates = ['text','article','content','body','Text','Article','Content','Body']
label_col_candidates = ['category','label','Category','Label','topic','Topic','class','Class']

def pick_col(cands, cols):
    for c in cands:
        if c in cols:
            return c
    return None

TEXT_COL = pick_col(text_col_candidates, df_raw.columns)
LABEL_COL = pick_col(label_col_candidates, df_raw.columns)

if TEXT_COL is None or LABEL_COL is None:
    raise ValueError("Could not infer TEXT_COL or LABEL_COL. Please set them manually.")

df = df_raw[[TEXT_COL, LABEL_COL]].dropna().rename(columns={TEXT_COL:'text', LABEL_COL:'label'}).reset_index(drop=True)
df['label'] = df['label'].astype(str)

print("Detected text column as 'text' and label column as 'label'")
display(df.head(3))
print("Class distribution:\n", df['label'].value_counts())

# Download the stopwords corpus if not already present
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Download the punkt_tab resource if not already present
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

# Download the wordnet resource if not already present
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

url_pat = re.compile(r'https?://\S+|www\.\S+')
email_pat = re.compile(r'\S+@\S+')
handle_pat = re.compile(r'@\w+')
num_pat = re.compile(r'\b\d+\b')


lemmatizer = WordNetLemmatizer()

def clean_text(s):
    s = s if isinstance(s, str) else ""
    s = s.lower()
    s = url_pat.sub(' ', s)
    s = email_pat.sub(' ', s)
    s = handle_pat.sub(' ', s)
    s = num_pat.sub(' ', s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(s)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words and len(t) > 2]
    return " ".join(tokens)

df['text_clean'] = df['text'].apply(clean_text)


from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


X = df['text_clean']
y = df['label']

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

def train_eval(model, name, k_select=None):
    steps = [('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, sublinear_tf=True))]
    if k_select:
        steps.append(('select', SelectKBest(chi2, k=k_select)))
    steps.append(('clf', model))
    pipe = Pipeline(steps)
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    acc = accuracy_score(y_te, preds)
    f1m = f1_score(y_te, preds, average='macro')
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f} | Macro-F1: {f1m:.4f}")
    print(classification_report(y_te, preds))
    return name, acc, f1m

results = []
results.append(train_eval(LogisticRegression(max_iter=2000, C=2.0), "LogisticRegression"))
results.append(train_eval(LinearSVC(), "LinearSVC"))
results.append(train_eval(MultinomialNB(), "MultinomialNB"))
results.append(train_eval(RandomForestClassifier(n_estimators=300, random_state=42), "RandomForest"))
results.append(train_eval(LinearSVC(), "LinearSVC + SelectKBest(20k)", k_select=20000))

res_df = pd.DataFrame(results, columns=['Model','Accuracy','MacroF1']).sort_values('MacroF1', ascending=False)
print("\nModel comparison:")
display(res_df)

In [ ]:

import itertools, collections

N = min(1000, len(df))
ner_docs = list(nlp.pipe(df['text'].iloc[:N].tolist()))
labels = df['label'].iloc[:N].tolist()

wanted = {"PERSON","ORG","GPE","DATE","MONEY"}
records = []
for doc, lab in zip(ner_docs, labels):
    for ent in doc.ents:
        if ent.label_ in wanted:
            records.append({"entity": ent.text, "etype": ent.label_, "label": lab})

ner_df = pd.DataFrame(records)
print("NER head:")
display(ner_df.head())

ent_freq = ner_df.groupby(['etype','entity']).size().reset_index(name='count').sort_values('count', ascending=False)
print("Top entities overall:")
display(ent_freq.head(20))

cross = ner_df.groupby(['label','etype']).size().unstack(fill_value=0)
print("Entity-type frequency by news category:")
display(cross)

co_pairs = collections.Counter()
for doc in ner_docs:
    ents = [e.text for e in doc.ents if e.label_ in wanted]
    for a,b in itertools.combinations(sorted(set(ents)), 2):
        co_pairs[(a,b)] += 1

pair_df = pd.DataFrame(
    [(a,b,c) for (a,b),c in co_pairs.most_common(25)],
    columns=['entity_a','entity_b','co_count']
)
print("Top co-occurring entity pairs:")
display(pair_df)


In [ ]:

pos_words = set(["good","great","excellent","positive","success","gain","benefit","win"])
neg_words = set(["bad","poor","terrible","negative","loss","decline","fail","risk"])

def rule_sent(s):
    toks = s.lower().split()
    score = sum(1 for t in toks if t in pos_words) - sum(1 for t in toks if t in neg_words)
    return 1 if score >= 0 else 0  # 1=pos, 0=neg (toy)

sample_texts = [
    "Markets show strong gain and positive outlook today.",
    "Company reports terrible loss and poor performance.",
    "Mixed results with moderate risk and some benefit."
]
print("Rule-based sentiment (toy):")
for t in sample_texts:
    print(rule_sent(t), "—", t)
